<br>
<center>
<img src="https://raw.githubusercontent.com/elaynelemos/prediction-of-orders-dmc/main/assets/img/univasf-logo.png" width=200>
<h3>
    UNIVERSIDADE FEDERAL DO VALE DO SÃO FRANCISCO
    <br>COLEGIADO DE ENGENHARIA DE COMPUTAÇÃO
</h3>

<h3>Orientador</h3>
<span>Prof. Dr. Rosalvo Ferreira de Oliveira Neto</span>

<h3>Discentes</h3>
<span>Anísio Pereira Batista Filho
<br>Edjair Aguiar Gomes Filho
<br>Elayne Rute Lessa Lemos</span>
</center>
<br><br>

## Regressão com Redes Neurais

Projeto em: [https://github.com/anisiobfilho/regression_neuralnetworks_univasf](https://github.com/anisiobfilho/regression_neuralnetworks_univasf)

### Importação das bibliotecas

In [1]:
#OS
import os
#Numpy
import numpy as np
#Pandas
import pandas as pd
#Matplotlib
import matplotlib.pyplot as plt

### Importação das bases de dados

In [2]:
repo_url = 'https://raw.githubusercontent.com/anisiobfilho/regression_neuralnetworks_univasf/main'

In [3]:
ic_house_pred_train = pd.read_csv(f'{repo_url}/data/ic_house_pred_train.csv')
ic_house_pred_train.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
274,435,180,RM,21.0,1890,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,81000
218,1231,90,RL,NaN,18890,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,8300,8,2007,WD,Normal,190000
1010,1397,20,RL,NaN,57200,Pave,NaN,IR1,Bnk,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,160000
348,558,50,C (all),60.0,11040,Pave,NaN,Reg,Low,AllPub,...,0,NaN,NaN,NaN,0,9,2006,COD,Normal,108000
23,439,30,RL,40.0,4280,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2007,WD,Normal,90350


In [4]:
ic_house_pred_test = pd.read_csv(f'{repo_url}/data/ic_house_pred_test.csv')
ic_house_pred_test.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
38,1033,60,RL,NaN,14541,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,11,2006,WD,Abnorml,310000
334,579,160,FV,34.0,3604,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Abnorml,146000
411,705,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,213000
82,423,20,RL,100.0,21750,Pave,NaN,Reg,HLS,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,113000
68,1132,20,RL,63.0,10712,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,9,2007,Oth,Abnorml,93500


### Separação da base em features e variável alvo

In [5]:
ic_house_pred_train_X = ic_house_pred_train.drop(columns='SalePrice')
ic_house_pred_train_X.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
963,1364,60,RL,73.0,8499,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,3,2007,New,Partial
657,774,20,RL,70.0,10150,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,7,2007,COD,Normal
388,1336,20,RL,80.0,9650,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2009,WD,Normal
1011,88,160,FV,40.0,3951,Pave,Pave,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2009,New,Partial
974,167,20,RL,NaN,10708,Pave,NaN,IR1,Lvl,AllPub,...,142,0,NaN,GdWo,NaN,0,11,2009,COD,Normal


In [6]:
ic_house_pred_train_y = pd.DataFrame(ic_house_pred_train['SalePrice'], columns=['SalePrice'])
ic_house_pred_train_y.sample(5)

,SalePrice
294,204750
398,111000
149,160000
844,118000
557,184000


In [7]:
ic_house_pred_test_X = ic_house_pred_test.drop(columns='SalePrice')
ic_house_pred_test_X.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
363,1058,60,RL,NaN,29959,Pave,NaN,IR2,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,1,2009,WD,Normal
176,240,50,RL,52.0,8741,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,4,2010,WD,Normal
134,917,20,C (all),50.0,9000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,10,2006,WD,Abnorml
367,1385,50,RL,60.0,9060,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,10,2009,WD,Normal
334,579,160,FV,34.0,3604,Pave,Pave,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Abnorml


In [8]:
ic_house_pred_test_y = pd.DataFrame(ic_house_pred_test['SalePrice'], columns=['SalePrice'])
ic_house_pred_test_y.sample(5)

,SalePrice
72,109008
270,465000
345,310000
71,134000
319,87500


### Eliminar a variável irrelevante PID (Nominal): Parcel identification number

In [9]:
ic_house_pred_train_X.drop(columns='Id', inplace=True)
ic_house_pred_train_X.sample(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
661,50,RM,50.0,5500,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
72,20,RL,65.0,8773,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
708,45,RM,55.0,4388,Pave,NaN,IR1,Bnk,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2007,WD,Normal
655,70,RM,121.0,17671,Pave,Grvl,Reg,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal
926,60,RL,NaN,11170,Pave,NaN,IR2,Lvl,AllPub,Corner,...,0,0,NaN,GdPrv,NaN,0,4,2006,WD,Normal


In [10]:
ic_house_pred_test_X.drop(columns='Id', inplace=True)
ic_house_pred_test_X.sample(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
96,50,RL,55.0,7264,Pave,NaN,Reg,Lvl,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,10,2009,WD,Normal
2,30,RM,56.0,8960,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,3,2010,WD,Normal
359,20,RL,91.0,10437,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2008,WD,Normal
329,20,RL,71.0,9353,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,Shed,0,7,2006,Oth,Abnorml
85,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,NaN,NaN,0,7,2009,WD,Normal


### Tratamento dos Missing Values (Apenas colocar para a média)

In [11]:
# replace_missing_by_fixed_value() é baseado na solução proposta no Estudo de Caso
# do livro Ciência dos Dados pelo Processo de KDD do Prof. Dr. Rosalvo Neto
# livro em: https://www.researchgate.net/publication/352749819_Ciencia_dos_Dados_pelo_Processo_de_KDD
# implementação em: https://github.com/rosalvoneto/Livro

def replace_missing_by_fixed_value(df, value, features):
    for column in features:
        df[column].fillna(value, inplace=True)

    return df

In [12]:
# replace_missing_by_mean() é baseado na solução proposta no Estudo de Caso
# do livro Ciência dos Dados pelo Processo de KDD do Prof. Dr. Rosalvo Neto
# Livro em: https://www.researchgate.net/publication/352749819_Ciencia_dos_Dados_pelo_Processo_de_KDD
# implementação em: https://github.com/rosalvoneto/Livro

def replace_missing_by_mean(df, numeric_features):
    for column in numeric_features:
        average = df[column].mean(axis=0)
        df[column].fillna(average, inplace=True)
    
    return df

In [13]:
print(ic_house_pred_train_X.columns.values)

['MSSubClass' 'MSZoning' 'LotFrontage' 'LotArea' 'Street' 'Alley'
 'LotShape' 'LandContour' 'Utilities' 'LotConfig' 'LandSlope'
 'Neighborhood' 'Condition1' 'Condition2' 'BldgType' 'HouseStyle'
 'OverallQual' 'OverallCond' 'YearBuilt' 'YearRemodAdd' 'RoofStyle'
 'RoofMatl' 'Exterior1st' 'Exterior2nd' 'MasVnrType' 'MasVnrArea'
 'ExterQual' 'ExterCond' 'Foundation' 'BsmtQual' 'BsmtCond' 'BsmtExposure'
 'BsmtFinType1' 'BsmtFinSF1' 'BsmtFinType2' 'BsmtFinSF2' 'BsmtUnfSF'
 'TotalBsmtSF' 'Heating' 'HeatingQC' 'CentralAir' 'Electrical' '1stFlrSF'
 '2ndFlrSF' 'LowQualFinSF' 'GrLivArea' 'BsmtFullBath' 'BsmtHalfBath'
 'FullBath' 'HalfBath' 'BedroomAbvGr' 'KitchenAbvGr' 'KitchenQual'
 'TotRmsAbvGrd' 'Functional' 'Fireplaces' 'FireplaceQu' 'GarageType'
 'GarageYrBlt' 'GarageFinish' 'GarageCars' 'GarageArea' 'GarageQual'
 'GarageCond' 'PavedDrive' 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch'
 '3SsnPorch' 'ScreenPorch' 'PoolArea' 'PoolQC' 'Fence' 'MiscFeature'
 'MiscVal' 'MoSold' 'YrSold' 'SaleType' '

In [14]:

# FEATURES:
nominal_features = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LandContour', 
                    'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
                    'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 
                    'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'GarageType', 
                    'MiscFeature', 'SaleType', 'SaleCondition']

ordinal_features = ['LotShape', 'Utilities',  'LandSlope', 'OverallQual', 'OverallCond', 
                    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
                    'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'Electrical', 'KitchenQual', 
                    'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 
                    'PavedDrive', 'PoolQC', 'Fence']

discrete_features = ['YearBuilt', 'YearRemodAdd', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 
                     'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 
                     'GarageYrBlt', 'GarageCars', 'MoSold', 'YrSold']

continuous_features = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 
                       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 
                       'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 
                       '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']


print("LENGHT:\n\tNominal: %d\n\tOrdinal: %d\n\tDiscrete: %d\n\tContinuous: %d" % (len(nominal_features), len(ordinal_features), len(discrete_features), len(continuous_features)))

LENGHT:
	Nominal: 23
	Ordinal: 23
	Discrete: 14
	Continuous: 19


#### ic_house_pred_train

In [15]:
ic_house_pred_train_X.columns[ic_house_pred_train_X.isnull().any()]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [16]:
ic_house_pred_train_X = replace_missing_by_fixed_value(ic_house_pred_train_X, 'ausente', nominal_features)
ic_house_pred_train_X = replace_missing_by_fixed_value(ic_house_pred_train_X, 'ausente', ordinal_features)

ic_house_pred_train_X = replace_missing_by_mean(ic_house_pred_train_X, discrete_features)
ic_house_pred_train_X = replace_missing_by_mean(ic_house_pred_train_X, continuous_features)


In [17]:
ic_house_pred_train_X.columns[ic_house_pred_train_X.isnull().any()]

Index([], dtype='object')

In [18]:
ic_house_pred_train_X.sample(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
966,20,RL,48.000,10635,Pave,ausente,IR2,Lvl,AllPub,FR2,...,0,0,ausente,ausente,ausente,0,5,2010,WD,Normal
265,50,RM,70.375,7920,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,0,0,ausente,MnPrv,ausente,0,3,2008,WD,Normal
10,20,RL,70.375,7390,Pave,ausente,IR1,Lvl,AllPub,Inside,...,0,0,ausente,ausente,ausente,0,7,2008,WD,Normal
592,120,RL,50.000,8012,Pave,ausente,Reg,Lvl,AllPub,Inside,...,0,0,ausente,ausente,ausente,0,7,2007,WD,Normal
420,60,RL,65.000,8450,Pave,ausente,Reg,Lvl,AllPub,Inside,...,0,0,ausente,ausente,ausente,0,2,2008,WD,Normal


#### ic_house_pred_test

In [19]:
ic_house_pred_test_X.columns[ic_house_pred_test_X.isnull().any()]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'],
      dtype='object')

In [20]:
ic_house_pred_test_X = replace_missing_by_fixed_value(ic_house_pred_test_X, 'ausente', nominal_features)
ic_house_pred_test_X = replace_missing_by_fixed_value(ic_house_pred_test_X, 'ausente', ordinal_features)

ic_house_pred_test_X = replace_missing_by_mean(ic_house_pred_test_X, discrete_features)
ic_house_pred_test_X = replace_missing_by_mean(ic_house_pred_test_X, continuous_features)


In [21]:
ic_house_pred_test_X.columns[ic_house_pred_test_X.isnull().any()]

Index([], dtype='object')

In [22]:
ic_house_pred_test_X.sample(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
393,20,RL,90.0,11694,Pave,ausente,Reg,Lvl,AllPub,Inside,...,260,0,ausente,ausente,ausente,0,7,2007,New,Partial
271,120,RL,41.0,5395,Pave,ausente,IR1,HLS,AllPub,Inside,...,0,0,ausente,ausente,ausente,0,10,2008,WD,Normal
91,60,RL,80.0,10793,Pave,ausente,Reg,Lvl,AllPub,Inside,...,0,0,ausente,GdWo,ausente,0,4,2007,WD,Normal
320,20,RL,79.0,9541,Pave,ausente,IR1,Lvl,AllPub,Inside,...,0,0,ausente,ausente,ausente,0,9,2009,New,Partial
401,60,RL,52.0,46589,Pave,ausente,IR2,Lvl,AllPub,CulDSac,...,0,0,ausente,ausente,ausente,0,7,2009,WD,Normal


### Criação de variáveis Dummies

In [23]:
ic_house_pred_train_X = pd.get_dummies(ic_house_pred_train_X, prefix_sep ='_', drop_first = True)
ic_house_pred_train_X.sample(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
482,70,60.0,7200,7,6,1931,2000,0.0,0,0,...,0,0,0,0,1,0,0,0,1,0
663,160,24.0,2368,5,6,1970,1970,312.0,765,0,...,0,0,0,0,1,0,0,0,1,0
166,20,83.0,10420,6,5,2009,2009,0.0,36,0,...,0,0,0,0,1,0,0,0,1,0
125,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,0,1,0,0,0,0,0
132,20,78.0,10140,6,5,1974,1974,174.0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [24]:
ic_house_pred_test_X = pd.get_dummies(ic_house_pred_test_X, prefix_sep ='_', drop_first = True)
ic_house_pred_test_X.sample(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
381,190,69.317073,32463,4,4,1961,1975,149.0,1159,0,...,0,0,0,0,0,1,0,0,1,0
36,50,60.000000,7200,5,4,1950,1982,0.0,180,352,...,0,0,0,0,0,1,0,0,1,0
110,30,51.000000,6120,2,3,1936,1950,0.0,0,0,...,0,0,1,0,0,0,0,0,1,0
140,60,174.000000,15138,8,5,1995,1996,506.0,689,0,...,0,0,0,0,0,1,0,0,1,0
377,20,105.000000,11751,6,6,1977,1977,480.0,705,0,...,0,0,0,0,0,0,0,0,1,0


In [25]:
# O código abaixo é baseado na solução proposta no Estudo de Caso
# do livro Ciência dos Dados pelo Processo de KDD do Prof. Dr. Rosalvo Neto
# livro em: https://www.researchgate.net/publication/352749819_Ciencia_dos_Dados_pelo_Processo_de_KDD
# implementação em: https://github.com/rosalvoneto/Livro

# Verificar as variáveis que estão no conjunto de treinamento e não estão no conjunto de teste;
# Verificar as variáveis que estão no conjunto de teste e não estão no conjunto de treinamento.

var_TRN = list(ic_house_pred_train_X.columns)
var_TST = list(ic_house_pred_test_X.columns)

In_TRN_Not_TST = np.setdiff1d(var_TRN, var_TST)
In_TST_Not_TRN = np.setdiff1d(var_TST, var_TRN)

if len(In_TRN_Not_TST)>0:
    print('Variáveis que estão apenas no Treinamento:\n')
    print(In_TRN_Not_TST)
else:
    print('Todas as variáveis do Treinamento estão no Teste!')
    
if len(In_TST_Not_TRN)>0:
    print('\nVariáveis que estão apenas no Teste:\n')
    print(In_TST_Not_TRN)
else:
    print('\nTodas as variáveis do Teste estão no Treinamento!')

Variáveis que estão apenas no Treinamento:

['Condition1_RRNn' 'Condition2_Feedr' 'Condition2_PosN' 'Condition2_RRAe'
 'Condition2_RRAn' 'Electrical_FuseP' 'Electrical_ausente' 'ExterCond_Po'
 'Exterior1st_AsphShn' 'Exterior1st_BrkComm' 'Exterior1st_CBlock'
 'Exterior2nd_CBlock' 'Exterior2nd_Other' 'Foundation_Wood'
 'Functional_Sev' 'GarageCond_Fa' 'GarageQual_Fa' 'HeatingQC_Po'
 'Heating_GasA' 'Heating_OthW' 'MiscFeature_TenC' 'PoolQC_Fa'
 'RoofMatl_CompShg' 'RoofMatl_Metal' 'RoofMatl_Roll' 'RoofStyle_Shed'
 'SaleCondition_AdjLand' 'SaleType_CWD' 'SaleType_Con' 'Utilities_NoSeWa']

Variáveis que estão apenas no Teste:

['Condition2_PosA' 'Condition2_RRNn' 'Electrical_Mix'
 'Exterior1st_ImStucc' 'RoofMatl_Membran']


In [26]:
# Adicionando as colunas faltantes em ambos os Dataframes

for column_name in In_TRN_Not_TST:
    ic_house_pred_test_X[column_name] = 0
    
for column_name in In_TST_Not_TRN:
    ic_house_pred_train_X[column_name] = 0

In [27]:
ic_house_pred_train_X.sample(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Condition2_PosA,Condition2_RRNn,Electrical_Mix,Exterior1st_ImStucc,RoofMatl_Membran
240,120,50.000,8012,6,5,1993,1994,0.0,165,841,...,0,0,0,1,0,0,0,0,0,0
251,75,53.000,7128,7,5,1941,1950,0.0,364,0,...,0,0,0,1,0,0,0,0,0,0
406,80,70.375,9125,7,5,1992,1992,170.0,0,0,...,0,0,0,1,0,0,0,0,0,0
787,30,50.000,8765,4,6,1936,1950,0.0,285,0,...,0,0,0,0,0,0,0,0,0,0
154,60,70.375,11075,5,4,1969,1969,232.0,562,193,...,0,0,0,1,0,0,0,0,0,0


In [28]:
ic_house_pred_test_X.sample(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,MiscFeature_TenC,PoolQC_Fa,RoofMatl_CompShg,RoofMatl_Metal,RoofMatl_Roll,RoofStyle_Shed,SaleCondition_AdjLand,SaleType_CWD,SaleType_Con,Utilities_NoSeWa
10,50,52.0,6292,7,7,1928,1950,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,20,50.0,6000,5,5,1948,1950,0.0,104,169,...,0,0,0,0,0,0,0,0,0,0
215,20,75.0,9000,8,5,2008,2008,36.0,1078,0,...,0,0,0,0,0,0,0,0,0,0
201,60,57.0,21872,7,5,1996,1997,0.0,604,0,...,0,0,0,0,0,0,0,0,0,0
346,20,72.0,11846,9,5,2003,2004,562.0,1567,0,...,0,0,0,0,0,0,0,0,0,0


### Normalização das variáveis

In [29]:
# uniformização de valores numéricos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

ic_house_pred_train_X_norm = scaler.fit_transform(ic_house_pred_train_X)
ic_house_pred_test_X_norm = scaler.fit_transform(ic_house_pred_test_X)

In [30]:
ic_house_pred_train_X[ic_house_pred_train_X.columns] = ic_house_pred_train_X_norm
ic_house_pred_train_X.sample(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Condition2_PosA,Condition2_RRNn,Electrical_Mix,Exterior1st_ImStucc,RoofMatl_Membran
551,0.000000,0.136986,0.041721,0.777778,0.500,0.985507,0.966667,0.195936,0.000000,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
759,0.823529,0.010274,0.000000,0.555556,0.625,0.782609,0.500000,0.000000,0.050496,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
284,0.058824,0.099315,0.037580,0.333333,0.625,0.500000,0.000000,0.000000,0.060950,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
38,1.000000,0.184932,0.048260,0.444444,0.375,0.673913,0.250000,0.000000,0.149008,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
966,0.000000,0.092466,0.043633,0.777778,0.500,0.949275,0.883333,0.124093,0.065556,0.862467,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
ic_house_pred_test_X[ic_house_pred_test_X.columns] = ic_house_pred_test_X_norm
ic_house_pred_test_X.sample(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,MiscFeature_TenC,PoolQC_Fa,RoofMatl_CompShg,RoofMatl_Metal,RoofMatl_Roll,RoofStyle_Shed,SaleCondition_AdjLand,SaleType_CWD,SaleType_Con,Utilities_NoSeWa
345,0.235294,0.418301,0.137318,1.000,0.428571,0.992537,0.966667,0.100000,0.349634,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
310,0.235294,0.352941,0.093244,0.500,0.571429,0.761194,0.450000,0.087500,0.244516,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
362,0.235294,0.315798,0.113816,0.750,0.428571,0.947761,0.866667,0.093125,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209,0.000000,0.315798,0.115288,0.375,0.428571,0.679104,0.266667,0.040625,0.155393,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252,0.176471,0.315798,0.110481,0.375,0.428571,0.485075,0.533333,0.000000,0.182358,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Transformação da variável alvo com a função logaritmo natural np.log(x)

In [32]:
ic_house_pred_train_y_log = pd.DataFrame(np.log(ic_house_pred_train_y['SalePrice']), columns=['SalePrice'])
ic_house_pred_train_y_log.sample(5)

,SalePrice
582,12.305918
471,12.160029
505,11.951180
680,12.061047
860,12.016122


In [33]:
ic_house_pred_test_y_log = pd.DataFrame(np.log(ic_house_pred_test_y['SalePrice']), columns=['SalePrice'])
ic_house_pred_test_y_log.sample(5)

,SalePrice
190,11.715866
276,11.831379
31,12.556730
218,11.775290
436,11.608236


### Exportar duas bases de dados:

In [34]:
path = 'data/preprocessed'
if not os.path.exists(path):
    os.mkdir(path)

ic_house_pred_train_X.to_csv(f'{path}/ic_house_pred_train_X.csv', index=False)
ic_house_pred_train_y.to_csv(f'{path}/ic_house_pred_train_y.csv', index=False)
ic_house_pred_train_y_log.to_csv(f'{path}/ic_house_pred_train_y_log.csv', index=False)

ic_house_pred_test_X.to_csv(f'{path}/ic_house_pred_test_X.csv', index=False)
ic_house_pred_test_y.to_csv(f'{path}/ic_house_pred_test_y.csv', index=False)
ic_house_pred_test_y_log.to_csv(f'{path}/ic_house_pred_test_y_log.csv', index=False)
